In [1]:
from c4 import Board
from c4 import Player
from c4 import Game
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy



def flatten_game_board_state(board):
    flat = np.append(board.rows[1:-1, 1:-1].flatten(), board.victory)

    output = np.array([f if f < 2 else -1 for f in flat])
    output[-1] += 1

    return output

def expand_board_state(flattened_state):
    '''This expands a 1x43 saved game state into a board.'''
    board, victory = np.array(flattened_state[:-1]), flattened_state[-1]

def generate_training_data(games, verbose = False):
    '''This function plays a set number of games, and returns
    the results in a format suitable for tensorflow.  Each space on the
    6x7 board contains an element in [-1, 0, 1] where 1 is the current users,
    0 is an empty space, and -1 is the opponent.  This matrix is flattened
    into one dimension and the resulting game winner (or 0 for a draw) is appended
    to the end so that each game result is represented by a 1x43 array.'''
    game_boards = []
    # results = []

    p1, p2 = Player(), Player()
    game = Game(p1, p2)

    for _ in range(games):
        game.start()
        game_boards.append(flatten_game_board_state(game.board))
        # results.append(self.board.victory)

    return game_boards #, results

def store_training_data(game_boards):
    with open('game_data.csv', 'a') as f:
        for result in game_boards:
            f.write(json.dumps(result.tolist()) + "\n")


2023-08-23 10:45:21.584283: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-23 10:45:22.189844: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-23 10:45:22.191891: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 10:45:30.990846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import numpy as np
import json
def expand_board_state(flattened_state):
    '''This expands a 1x43 saved game state into a board.'''

    board, victory = flattened_state[:-1], flattened_state[-1]
    board = np.reshape(board, (6,7))
    
    return board, victory

def get_training_data(test_percentage = 20):
    with open('game_data.csv', 'r') as f:
        data = f.readlines()
        data = [json.loads(d) for d in data]

    x_test, y_test, x_train, y_train = [], [], [], []
    
    cutoff= (len(data) * test_percentage)// 100
    
    for d in data[:-cutoff]:
        x_test.append(np.array([d[:-1]]))
        y_test.append(d[-1])
        
    for d in data[-cutoff:]:
        x_train.append(np.array([d[:-1]]))
        y_train.append(d[-1])

    x_test = np.array(x_test)
    y_test = np.array(y_test)
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    return x_test, y_test, x_train, y_train


def train(model):
    p1 = Player("AI")
    p2 = Player("AI")
    game = Game(p1, p2)

#     game_results = generate_training_data(10000)
#     store_training_data(game_results)
    
#     x_train = []
#     y_train = []
    
    
#     for result in game_results:
#         x_train.append([result[:-1]])
#         y_train.append(result[-1])

#     x_train = np.array(x_train)
#     y_train = np.array(y_train)
    
        
#     print(x_train[0])

    x_train, y_train, x_test, y_test = get_training_data()
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    print(len(x_train))
    model.fit(x_train, y_train, epochs=5)

    return model



def build_model():
    '''This creates our tensorflow model'''
    model = Sequential()
    model.add(Flatten(input_shape=(1, 42)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='linear'))
    
    return model

def predict(x_test, y_test):
    pass

model = build_model()
train(model).summary()
model.predict([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 1, 0, -1, 0, 0, 0, -1, 1, 1, 1, 0, 0, 0, -1, 1, 1, 1, -1, 0, 1, -1, -1, -1, 1, -1]],
[[1, 1, 1, -1, -1, 0, 0, -1, -1, 1, 1, 1, -1, -1, -1, 1, -1, 1, -1, 1, 1, 1, 1, -1, -1, -1, 1, -1, 1, -1, -1, -1, 1, -1, 1, 1, 1, 1, -1, -1, -1, 1]]])

80000
Epoch 1/5


/home/alex/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(32, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


2500/2500 [==============================] - 3s 1ms/step - loss: 1.3305e-07 - accuracy: 0.4164
Epoch 2/5
2500/2500 [==============================] - 3s 1ms/step - loss: 1.3305e-07 - accuracy: 0.4164
Epoch 3/5
2500/2500 [==============================] - 3s 1ms/step - loss: 1.3305e-07 - accuracy: 0.4164
Epoch 4/5
2500/2500 [==============================] - 3s 1ms/step - loss: 1.3305e-07 - accuracy: 0.4164
Epoch 5/5
2500/2500 [==============================] - 3s 1ms/step - loss: 1.3305e-07 - accuracy: 0.4164
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 42)                0         
                                                                 
 dense_8 (Dense)             (None, 100)               4300      
                                                                 
 dense_9 (Dense)             (None, 100)               10100     
 

array([[-0.23385796],
       [ 0.11969101]], dtype=float32)

In [3]:
import numpy as np
import json
def expand_board_state(flattened_state):
    '''This expands a 1x43 saved game state into a board.'''

    board, victory = flattened_state[:-1], flattened_state[-1]
    board = np.reshape(board, (6,7))
    
    return board, victory

def get_training_data(test_percentage = 20):
    with open('game_data.csv', 'r') as f:
        data = f.readlines()
        data = [json.loads(d) for d in data]

    x_test, y_test, x_train, y_train = [], [], [], []
    
    cutoff= (len(data) * test_percentage)// 100
    
    for d in data[:-cutoff]:
        x_test.append(np.array([d[:-1]]))
        y_test.append(d[-1])
        
    for d in data[-cutoff:]:
        x_train.append(np.array([d[:-1]]))
        y_train.append(d[-1])

    x_test = np.array(x_test)
    y_test = np.array(y_test)
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    return x_test, y_test, x_train, y_train
    
x_test, y_test, x_train, y_train = get_training_data()

In [65]:
x_test[-1], y_test[-1]

(array([[ 0,  1, -1,  0,  0,  0,  0,  0, -1, -1,  1, -1,  0,  0,  0,  1,
         -1, -1,  1,  0,  0,  0,  1, -1,  1, -1,  0,  0,  0,  1,  1,  1,
         -1,  0,  0,  0, -1,  1,  1, -1,  0,  0]]),
 0)

In [7]:
from c4 import Board
from c4 import Player
from c4 import Game
import json
import numpy as np



def flatten_game_board_state(board):
    flat = np.append(board.rows[1:-1, 1:-1].flatten(), board.victory)

    output = np.array([f if f < 2 else -1 for f in flat])

    return output

def expand_board_state(flattened_state):
    '''This expands a 1x43 saved game state into a board.'''

    board, victory = np.array(flattened_state[:-1]), flattened_state[-1]

def generate_training_data(games, verbose = False):
    '''This function plays a set number of games, and returns
    the results in a format suitable for tensorflow.  Each space on the
    6x7 board contains an element in [-1, 0, 1] where 1 is the current users,
    0 is an empty space, and -1 is the opponent.  This matrix is flattened
    into one dimension and the resulting game winner (or 0 for a draw) is appended
    to the end so that each game result is represented by a 1x43 array.'''
    game_boards = []
    # results = []

    p1, p2 = Player(), Player()
    game = Game(p1, p2)

    for i in range(1, games + 1):
        game.start()
        game_boards.append(flatten_game_board_state(game.board))
        game_boards[-1][-1] += 1
        # results.append(self.board.victory)

        if verbose and i % 1000 == 0:
            print(f'{(i * 100)//games}% done; finished game: {i}')

    # if verbose:
    # 	for game, result in zip(game_boards, results):
    # 		print(result)
    # 		print(game)



    return game_boards #, results



def store_training_data(game_boards):
    with open('game_data.csv', 'a') as f:
        for result in game_boards:
            f.write(json.dumps(result.tolist()) + "\n")

def train():
    p1 = Player("AI")
    p2 = Player("AI")
    game = Game(p1, p2)

    game_results = generate_training_data(1000)

    store_training_data(game_results)

    build_model(game_results)

def build_model(game_results):
    '''This creates our tensorflow model'''
    model = Sequential()
    model.add(Flatten(input_shape=(1, 42)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(1, activation='linear'))
    return model

    x_train, y_train = [], []

    for result in game_results:
        x_train.append(result[:-1])
        y_train.append(result[-1])

    # model.compile(optimizer='adam',
    #           loss=loss_fn,
    #           metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=5)

    return model

def predict(x_test, y_test):
    pass




game_results = generate_training_data(40000, True)
store_training_data(game_results)
    

2% done; finished game: 1000
5% done; finished game: 2000
7% done; finished game: 3000
10% done; finished game: 4000
12% done; finished game: 5000
15% done; finished game: 6000
17% done; finished game: 7000
20% done; finished game: 8000
22% done; finished game: 9000
25% done; finished game: 10000
27% done; finished game: 11000
30% done; finished game: 12000
32% done; finished game: 13000
35% done; finished game: 14000
37% done; finished game: 15000
40% done; finished game: 16000
42% done; finished game: 17000
45% done; finished game: 18000
47% done; finished game: 19000
50% done; finished game: 20000
52% done; finished game: 21000
55% done; finished game: 22000
57% done; finished game: 23000
60% done; finished game: 24000
62% done; finished game: 25000
65% done; finished game: 26000
67% done; finished game: 27000
70% done; finished game: 28000
72% done; finished game: 29000
75% done; finished game: 30000
77% done; finished game: 31000
80% done; finished game: 32000
82% done; finished g